In [ ]:
from Crypto.Util.number import *
from Crypto.Random import random
from Crypto.Cipher import AES
from Crypto.Util.number import inverse
from Crypto.Util.Padding import pad, unpad
import hashlib
from sage.groups.generic import bsgs

In [ ]:
## microtransmisions
!pip3 install pycrypto
from Crypto.Util.number import *
from Crypto.Random import random
from Crypto.Cipher import AES
from Crypto.Util.number import inverse
from Crypto.Util.Padding import pad, unpad
from sage.all import *
from sage.groups.generic import bsgs
import hashlib

# curve parameters 
a = 1
b = 4
p = 99061670249353652702595159229088680425828208953931838069069584252923270946291
E = EllipticCurve(GF(p),[a,b])

G = E(43190960452218023575787899214023014938926631792651638044680168600989609069200, 20971936269255296908588589778128791635639992476076894152303569022736123671173)

ax = 87360200456784002948566700858113190957688355783112995047798140117594305287669
bx = 6082896373499126624029343293750138460137531774473450341235217699497602895121
A = E.lift_x(ax)
B = E.lift_x(bx)

# print(E.order())
# print(G.order())
# print(A.order())

assert E.order() == G.order() and E.order() == A.order()

def dlp_solve(G,A):
    primes = []
    for i in G.order().factor():
        if (len(primes)<8):
            primes.append(i)
        else:
            break
    corresponding_dlp = [0]*len(primes)
    for i,(p_i,e_i) in enumerate(primes):
        for j in range(e_i):
            corresponding_dlp[i]+=bsgs(G*(G.order()//p_i),(A-(G*corresponding_dlp[i]))*(G.order()//(p_i**(j+1))),(0,p_i),operation='+')*(p_i**j)
    return crt(corresponding_dlp,[p_i**e_i for (p_i,e_i) in primes])

n_a = dlp_solve(G,A)

def gen_shared_secret(P, n):
	S = n*P
	return S.xy()[0]

share_secret = gen_shared_secret(B,n_a)

def decrypt(shared_secret,encrypted_flag,iv):
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    cipher = AES.new(key,AES.MODE_CBC,iv)
    return cipher.decrypt(encrypted_flag)

print(share_secret)
iv = "ceb34a8c174d77136455971f08641cc5"
encrypted_flag = "b503bf04df71cfbd3f464aec2083e9b79c825803a4d4a43697889ad29eb75453"
print(decrypt(share_secret,bytes.fromhex(encrypted_flag),bytes.fromhex(iv)))

In [ ]:

## Elliptic Nodes
p = 4368590184733545720227961182704359358435747188309319510520316493183539079703

gx = 8742397231329873984594235438374590234800923467289367269837473862487362482
gy = 225987949353410341392975247044711665782695329311463646299187580326445253608
qx = 2582928974243465355371953056699793745022552378548418288211138499777818633265
qy = 2421683573446497972507172385881793260176370025964652384676141384239699096612

a = ((pow(gy,2,p) - pow(qy,2,p) - pow(gx,3,p) + pow(qx,3,p))*(pow(gx-qx,-1,p)))%p 
b = (pow(gy,2,p) - pow(gx,3,p) - a*gx)%p
c = (pow(qy,2,p) - pow(qx,3,p) - a*qx)%p
assert pow(gy,2,p) == (pow(gx,3,p) + a*gx + b)%p
assert pow(qy,2,p) == (pow(qx,3,p) + a*qx + b)%p

## Realise that the elliptic curve is singular
# E = EllipticCurve(GF(p),[a,b])

## Check if that's true using 4A^3 + 27B^2 = 0
# print((4*a**3+27*b**2)%p)
# P.<x> = PolynomialRing(GF(p),x)
# f = x**3 + a*x + b
# print(f.roots())

## Discrete Logarithm Problem for Singular Curves

P.<x> = PolynomialRing(GF(p),x)
f = x**3 + a*x + b
P = (gx, gy)
Q = (qx, qy)
## to calculate the affine shifts, execute print(f.factors())
f_ = f.subs(x=x + 1557923326969252180825193218688702224840389936248863823173183835359957757721)
P_ = (P[0] - 1557923326969252180825193218688702224840389936248863823173183835359957757721, P[1])
Q_ = (Q[0] - 1557923326969252180825193218688702224840389936248863823173183835359957757721, Q[1])
d = 1557923326969252180825193218688702224840389936248863823173183835359957757721 - 1252743530795041358577574745326954908754967315811591864173948822463623564261
t = GF(p)(d).square_root()

u = (P_[1] + t*P_[0])/(P_[1] - t*P_[0]) % p
v = (Q_[1] + t*Q_[0])/(Q_[1] - t*Q_[0]) % p
print(long_to_bytes(discrete_log(v, u)))


In [ ]:
## Moving Problems 
# Define Curve params
p = 1331169830894825846283645180581
a = -35
b = 98
E = EllipticCurve(GF(p), [a,b])
G = E.gens()[0]

P_a = E(1110072782478160369250829345256, 800079550745409318906383650948)
P_b = E(1290982289093010194550717223760, 762857612860564354370535420319)

F = EllipticCurve(GF(p**2),[a,b])
print(G.order())
print(P_a.order())

def dlp_solve(G,A):
    primes = []
    for i in G.order().factor():
        if (len(primes)<8):
            primes.append(i)
        else:
            break
    corresponding_dlp = [0]*len(primes)
    for i,(p_i,e_i) in enumerate(primes):
        for j in range(e_i):
            corresponding_dlp[i]+=bsgs(G*(G.order()//p_i),(A-(G*corresponding_dlp[i]))*(G.order()//(p_i**(j+1))),(0,p_i),operation='+')*(p_i**j)
    return crt(corresponding_dlp,[p_i**e_i for (p_i,e_i) in primes])
print(dlp_solve(G,P_a))



